## Building an ARIMA model 


In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
from statsmodels.tsa import stattools

In [ ]:
# load your dataset
data_dir = os.path.join(os.getcwd(), 'data_files')

target_df = pd.read_csv(os.path.join(data_dir, 'targets.csv'), parse_dates=['USAGE_DATE'], index_col='USAGE_DATE')
predictor_df = pd.read_csv(os.path.join(data_dir, 'predictors.csv'), parse_dates=['USAGE_DATE'], index_col='USAGE_DATE')
predictor_df['Holiday'] = predictor_df['Holiday'].astype(int)

df = target_df.join(predictor_df).sort_index(ascending=True)

In [ ]:
# identify whether target series is stationary by checking acf plot
def acf_plots(data, nlags, seasonal_m):
    acf_values = []
    for k in np.arange(0,nlags+1):
        acf_values.append(data.diff(seasonal_m).dropna().autocorr(lag=k))
    acf_5cv = 1.96/np.sqrt(len(df['SoCal_TOTAL']) - np.arange(0, nlags+1))
    pacf_values = stattools.pacf(data, nlags=nlags)
    
    print(len(acf_values), len(pacf_values))
    
    fig = plt.figure(figsize=(10, 8))
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])

    sns.lineplot(data=data, ax=ax1)

    pd.plotting.autocorrelation_plot(data, ax=ax2)
    ax2.set_title('Original ACF'), ax2.set_ylim([-1, 1]), ax2.set_xlim([0, nlags])

    sns.lineplot(data=pacf_values, ax=ax3)
    ax3.plot(np.arange(0, nlags+1), acf_5cv, 'k--')
    ax3.plot(np.arange(0, nlags+1), acf_5cv*-1, 'k--')
    ax3.set_title('PACF'), ax3.set_xlabel('Lag'), ax3.set_xlim([0, nlags])

    plt.tight_layout(), plt.show()
    
    marks_df = pd.DataFrame({'ACF': acf_values,
                             'PACF': pacf_values,
                             '5CV': acf_5cv,})
    
    adf_p = stattools.adfuller(data)[1]
    print('stationary if sig:', adf_p)
    return marks_df

In [ ]:
acf_plots(df['SoCal_TOTAL'], nlags=100, seasonal_m=365) #not sig
seasonal_diff =df['SoCal_TOTAL'].diff(365).dropna()
marks_df = acf_plots(seasonal_diff, nlags=100, seasonal_m=365) #sig

In [ ]:
np.diff(marks_df.index[marks_df['PACF']>marks_df['5CV']]).cumsum()

In [ ]:
# find D via Canova Hanson test to minimize search params for auto_arima seasonal component
pm.arima.utils.nsdiffs(df['SoCal_TOTAL'], 
                      m=365,
                      max_D=12,
                      test='ch')

In [ ]:
# NOTE: Seasonal order cannot exceed 200. For daily data with yearly seasonality (i.e. m=365) 
# seasonality needs to be resolve via FFT added on top of an ARIMA process
# see note in https://robjhyndman.com/hyndsight/longseasonality/

df_test = df[df.index >= pd.to_datetime('4/15/21')] 
df_train = df[df.index < pd.to_datetime('4/15/21')]
arima_fit = pm.auto_arima(df_train['SoCal_TOTAL'], 
                          start_p=25, max_p=33,
                          d=0, 
                          seasonal=True, m=30,
                          start_P=3, max_P=7,
                          D=1, 
                          stepwise=True)
arima_fit.summary()

In [ ]:
days_to_forecast = pd.date_range(start=df_train.index.max(), end = df_train.index.max() + pd.DateOffset(months=6), freq='1d')
predictions = pd.DataFrame({'ARIMA_predict': arima_fit.predict(len(days_to_forecast))})
predictions.index=days_to_forecast

compare_df = pd.concat([df_test['SoCal_TOTAL'], predictions], axis=1).rename(columns={'SoCal_TOTAL': 'Test'})
sns.lineplot(data=df_train['SoCal_TOTAL'], color='k', label='Train')
sns.lineplot(data=compare_df, )
plt.xlim([df_train.index.max() - pd.DateOffset(months=1), predictions.index.max()])
plt.show()

In [ ]:
# transform target data to stationary

lb_test = sm.stats.acorr_ljungbox(df[forecast_column+'_residuals'], lags=[27])

df['SoCal_TOTAL']